In [ ]:
import urllib.request                   # Library to fetch webpages
from bs4 import BeautifulSoup           # Library to parse data from webpages
from time import sleep
!pip install selenium
from selenium import webdriver 
import time
# consist of all the schema of the data we are extracting
from schema import SCHEMA
import pandas as pd
import selenium
import numpy as np

In [ ]:
login = {
    "username": "kanevi2020@sweatmail.com",
    "password": "testpass123"
}

In [ ]:
SCHEMA

In [ ]:
# accessing chrome webdriver
def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
#     install chromedriver in python directory and add path here
    browser = webdriver.Chrome(options=chrome_options,executable_path=r"\\Programs\\Python 3.8\\chromedriver")
    return browser
browser = get_browser()

In [ ]:
# login to glassdoor webiste using login credential 
def login():
    print('loggging to', login["username"])
    url = 'https://www.glassdoor.com/profile/login_input.htm'
    
    browser.get(url)

    time.sleep(1)                           # Give Javascript time to render
    email_field = browser.find_element_by_id('userEmail')
    time.sleep(1)
    password_field = browser.find_element_by_id('userPassword')
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys(login["username"])
    password_field.send_keys(login["password"])
    submit_btn.click()

In [ ]:
res = pd.DataFrame([], columns=SCHEMA)

In [ ]:
# provide the link of the webpage from which we data need to be extracted

In [ ]:
amazon_url = 'https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036.htm'
browser.get(amazon_url)

In [ ]:
# extracts data from the mentioned class
def extract_from_page():

    def is_featured(review):
        try:
            review.find_element_by_class_name('featuredFlag')
            return True
        except selenium.common.exceptions.NoSuchElementException:
            return False
        
# extracts author inforamtion - employee status, location , Job role
    def extract_review(review):
        author = review.find_element_by_class_name('authorInfo')
        res = {}
        # import pdb;pdb.set_trace()
        for field in SCHEMA:
            res[field] = scrape(field, review, author)

        assert set(res.keys()) == set(SCHEMA)
        return res

    res = pd.DataFrame([], columns=SCHEMA)

    reviews = browser.find_elements_by_class_name('empReview')

    for review in reviews:
        if not is_featured(review):
            data = extract_review(review)
            res.loc[idx[0]] = data
        else:
            print('Discarding a featured review')
        idx[0] = idx[0] + 1
    return res


In [ ]:
# checking if there are more than one page for the company
def more_pages():
    try:
        next_ = browser.find_element_by_class_name('pagination__PaginationStyle__next')
        next_.find_element_by_tag_name('a')
        return True
    except selenium.common.exceptions.NoSuchElementException:
        return False
# extracting information in the next page using pagination tag in the page 
def go_to_next_page():
    next_ = browser.find_element_by_class_name(
        'pagination__PaginationStyle__next').find_element_by_tag_name('a')
    browser.get(next_.get_attribute('href'))
    time.sleep(1)
    page[0] = page[0] + 1

In [ ]:
def scrape(field, review, author):
# extracts date and time 
    def scrape_date(review):
        return review.find_element_by_tag_name(
            'time').get_attribute('datetime')

    def scrape_emp_title(review):
        if 'Anonymous Employee' not in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorJobTitle').text.split('-')[1]
            except Exception:
                res = np.nan
        else:
            res = np.nan
        return res

    def scrape_location(review):
        if 'in' in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorLocation').text
            except Exception:
                res = np.nan
        else:
            res = np.nan
        return res

    def scrape_status(review):
        try:
            res = author.text.split('-')[0]
        except Exception:
            res = np.nan
        return res

    def scrape_rev_title(review):
        return review.find_element_by_class_name('summary').text.strip('"')

    def expand_show_more(section):
        try:
            more_link = section.find_element_by_class_name('v2__EIReviewDetailsV2__continueReading')
            more_link.click()
        except Exception:
            pass

    def scrape_pros(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[0].find_elements_by_tag_name('p')[1].text
            #res = res.strip()
        except Exception:
            res = np.nan
        return res

    def scrape_cons(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[1].find_elements_by_tag_name('p')[1].text
        except Exception:
            res = np.nan
        return res


    def scrape_overall_rating(review):
        try:
            ratings = review.find_element_by_class_name('gdStars')
            overall = ratings.find_element_by_class_name(
                'rating').find_element_by_class_name('value-title')
            res = overall.get_attribute('title')
        except Exception:
            res = np.nan
        return res

    funcs = [
        scrape_date,
        scrape_emp_title,
        scrape_location,
        scrape_status,
        scrape_rev_title,
        scrape_pros,
        scrape_cons,
        scrape_overall_rating,
    ]
    
    fdict = dict((s, f) for (s, f) in zip(SCHEMA, funcs))

    return fdict[field](review)

In [ ]:
page = [1]
idx = [0]
res = pd.DataFrame()
reviews_df = extract_from_page()
res = res.append(reviews_df)
date_limit_reached = [False]

In [ ]:
# give the number of review required as length of results
while more_pages() and\
        len(res) < 20:
    go_to_next_page()
    reviews_df = extract_from_page()
    res = res.append(reviews_df)
    time.sleep(1)

end = time.time()

In [ ]:
res.to_csv('Amazon_Review_Output1.csv')